In [ ]:
options("scipen"=100, "digits"=4)
if(!require("readr")) install.packages("readr")
if(!require("rpart")) install.packages("rpart")
if(!require("rpart.plot")) install.packages("rpart.plot")
library("readr")
library("rpart")
library("rpart.plot")

<table style="width:98%;">
<colgroup>
<col style="width: 9%" />
<col style="width: 4%" />
<col style="width: 8%" />
<col style="width: 7%" />
<col style="width: 8%" />
<col style="width: 9%" />
<col style="width: 8%" />
<col style="width: 11%" />
<col style="width: 8%" />
<col style="width: 12%" />
<col style="width: 9%" />
</colgroup>
<thead>
<tr class="header">
<th style="text-align: left;">Alternate</th>
<th style="text-align: left;">Bar</th>
<th style="text-align: left;">Fri/Sat</th>
<th style="text-align: left;">Hungry</th>
<th style="text-align: left;">Patrons</th>
<th style="text-align: left;">Price</th>
<th style="text-align: left;">Raining</th>
<th style="text-align: left;">Reservation</th>
<th style="text-align: left;">Type</th>
<th style="text-align: left;">WaitEstimate</th>
<th style="text-align: left;">WillWait</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Expensive</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">French</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">30-60</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">10-30</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="odd">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Expensive</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">French</td>
<td style="text-align: left;">&gt;60</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Moderate</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Italian</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">None</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Moderate</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">&gt;60</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Expensive</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Italian</td>
<td style="text-align: left;">10-30</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">None</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">30-60</td>
<td style="text-align: left;">Yes</td>
</tr>
</tbody>
</table>

In [ ]:
url<-"https://docs.google.com/spreadsheets/d/e/2PACX-1vQvQ1hQJoj_3BmOV3uLWcDzwIw3GFhkS84NePBS57lnCmXSN86KU8kn4aQUE9rGsxJPMi32S55JBl9w/pub?gid=0&single=true&output=csv"
traindf<-read.csv(url, stringsAsFactor=TRUE)

Now let’s print out the training set to make sure we read it correctly:

In [ ]:
print(traindf)

In [ ]:
control = rpart.control(minsplit=1,maxdepth=10)
model <- rpart(WillWait~., data=traindf, method="class", control = control)
rpart.plot(model, type=4, extra = 1, digits=-2)

Create models for each individual predictor with maxdepth=1:w

In [ ]:
control = rpart.control(minsplit=1,maxdepth=5)
model <- rpart(WillWait~Hungry, data=traindf, method="class", control = control)
rpart.plot(model, type=4, extra = 1, digits=-2)